In [ ]:
import visa                             # Import PyVISA library
import time
#import VISAresourceExtentions
#import numpy as np

In [ ]:
# Initialization

rm = visa.ResourceManager()             # Create Resource Manager object
rs = rm.list_resources()                # Method to list the available resources
print(rs[0])                            
counter = rm.open_resource(rs[0])       # Assign the returned object to the instrument variable (i.e. counter)

In [ ]:
# Initial settings

print(counter.query('*IDN?'))           # Query the Identification string 
counter.write('*RST;*CLS')              # Reset the instrument, clear the Error queue
counter.timeout = 1e7                   # Acquisition timeout (ms) - set it higher than the acquisition time
counter.query('SYST:ERR?')              # Error Checking 

In [ ]:
#Basic settings

counter.write("FUNC 'FREQ 1'")                          # Setup for frequency measurement from channel A (1) 
counter.write('INP:LEV:AUTO OFF; :INP:LEV 0')           # Enable/Disable autotrigger on channel A; Trigger level (V)
counter.query('SYST:ERR?')                              # Error Checking                         

In [ ]:
# Calculate MEAN/MAX/MIN/SDEV/ADEV values (ascii format), only if 'CALC:AVER:STAT ON'

counter.write('CALC:AVER:STAT ON')                                        # Enable/disable statitics; 
counter.write('CALC:AVER:TYPE MEAN')                                      # Select the statistical function to be performed (MIN/MAX/MEAN/SDEV/ADEV)
counter.write('FORMAT:TINF ON; :FORMAT ASCII')                            # Read timestamp of each measurement (can also be omitted with CONF and MEAS); Redout in ASCII/REAL mode -> [freq (Hz), tstamp (sec)]
counter.write('SENSE:ACQ:APER 2e-8; :CALC:AVER:COUNT 1e3')                # Gate time (sec), minimum: 20ns - Dead time = 6-8 microsec; Set the number of samples to use in statistics sampling                                      
#time.sleep(1)                                                             # Wait (sec)                                                           

start = time.time()

for i in range(1):
    
    counter.query('INIT;*OPC?')                                           # Initialize acquisition; *OPC? query waits until the acquisition ends 
   
    #counter.write('CALC:DATA?')                                 
    #counter.read()
    #value = counter.query('CALC:DATA?')                                   # Fetch list with string characters as its elements
    #value = counter.query_ascii_values('CALC:DATA?')                      # Fetch ASCII value ( ValueError: 'could not convert string to float can occur' ), sometimes doesn't work when timestamp is printed (i.e. with MIN/MAX )         
    value = counter.query_ascii_values('CALC:DATA?', converter = 's')     # Fetch and Convert ASCII value into a string

    value[-1] = value[-1].split('\n')[0]                                  # Delete in the string every character after the number ( i.e. \n0E..)
    value[0] = float(value[0])                                            # Convert string into float number
    value[1] = float(value[1])                                            # Condition necessary when the last element of the list happens to be an empty string = '' 

    print(value)                                                          # List format -> [value, timestamp]
    
end = time.time()
print(end-start)                                                          # Total acquisition time (sec)
counter.query('SYST:ERR?')                                                # Error Checking 

In [ ]:
# Perform  M ('ARM:COUNT') block measurements of N ('TRIG:COUNT') samples each (Dead time = 6-8 microsec)
# Fetch the data of one acquisiton ( Total number of samples/measurements = M*N )

counter.write('CALC:AVER:STAT OFF')                     # Enable/disable statitics; 
counter.write('CAL:INT:AUTO OFF; :DISP:ENAB OFF')       # Enable/Disable reciprocal counter that uses an interpolating technique to increase the resolution (if OFF -> Dead time = 4 microsec); Enable/Disable the display (if OFF increase the GPIB speed)
counter.write('FORMAT:TINF ON; :FORMAT ASCII')          # Read timestamp of each measurement (can also be omitted with CONF and MEAS); Redout in ASCII/REAL mode -> [freq (Hz), tstamp (sec)]
counter.write('SENSE:ACQ:APER 2e-8')                    # Gate time (sec), minimum: 20ns - Dead time = 6-8 microsec; Set the number of samples to use in statistics sampling                                      
counter.write('TRIG:COUNT 1e3; :ARM:COUNT 1e1')         # Measure N samples inside one block (Triggerings); Number M of blocks (Armings) - Max(N*M) = 3.5e6 (Size of the memory)
counter.query('SYST:ERR?')                              # Error Checking  
#time.sleep(1)                                           # Wait (sec)

In [ ]:
# Fetch the data in one cycle 

counter.query('INIT;*OPC?')                                                       # Initialize acquisition; *OPC? query waits until the acquisition ends

start = time.time()

#counter.write('FETCH:ARR? 1e3')                   
#counter.read()                                           
#freqtimelist = counter.query('FETCH:ARR? 1e3')                                    # Fetch list with string characters as its elements
#freqtimelist = counter.query_ascii_values('FETCH:ARR? 1e3')                       # Fetch list of ASCII values ( ValueError: 'could not convert string to float can occur' )            
freqtimelist = counter.query_ascii_values('FETCH:ARR? 1e3', converter = 's')      # Fetch and Convert list of ASCII values into a list of strings (Maximum Fetchable samples = 1e4)
    
if  freqtimelist[-1] == '' :    del freqtimelist[-1]                              # Delete the last element of the list when it is an empty string = '' 

freqtimelist[-1] = freqtimelist[-1].split('\n')[0]                                # Delete in the last element of the list of strings every character after the number ( i.e. \n0E..)
freqtimelist = [float(i) for i in freqtimelist]                                   # Convert list of strings into float numbers

print(freqtimelist)

end = time.time()
print(end-start)                                                                  # Total acquisition time (sec)
counter.query('SYST:ERR?')                                                        # Error Checking  

In [ ]:
# Fetch the data over multiple cycles 

counter.query('INIT;*OPC?')                                                             # Initialize acquisition; *OPC? query waits until the acquisition ends

start = time.time()

for i in range(10):
        
    #counter.write('FETCH:ARR? 1e3')                   
    #counter.read()                                                  
    #freqtimelist = counter.query('FETCH:ARR? 1e3')                                     # Fetch list with string characters as its elements
    #freqtimelist = counter.query_ascii_values('FETCH:ARR? 1e3')                        # Fetch list of ASCII values ( ValueError: 'could not convert string to float can occur' )            
    freqtimelist = counter.query_ascii_values('FETCH:ARR? 1e3', converter = 's')       # Fetch and Convert list of ASCII values into a list of strings (Maximum Fetchable samples = 1e4)
    
    if  freqtimelist[-1] == '' :    del freqtimelist[-1]                               # Delete the last element of the list when it is an empty string = ''

    freqtimelist[-1] = freqtimelist[-1].split('\n')[0]                                 # Delete in the last element of the list of strings every character after the number ( i.e. \n0E..)
    freqtimelist = [float(i) for i in freqtimelist]                                    # Convert list of strings into float numbers                                 
        
    print(freqtimelist)

    if (i==0): freqtimeList = freqtimelist                                             # Create a bigger List appending all the fetched data lists during the loop 
    else:  freqtimeList = freqtimeList + freqtimelist 
        
    end = time.time()
    print(end-start)                                                                   # Single acquisition time (sec)
    
end = time.time()
print(end-start)                                                                       # Total acquisition time (sec)
counter.query('SYST:ERR?')                                                             # Error Checking  

In [ ]:
counter.write('*RST;*CLS') 

In [ ]:
counter.close()